In [1]:
% load_ext autoreload
% autoreload 2

In [2]:
import utils
import train
import cvae

In [3]:
train_iter, val_iter, test, DE, EN = utils.torchtext_extract(d=0, DATASET="WMT14")

In [4]:
print(len(train_iter), len(val_iter))

38929 35


In [ ]:
model_name = "test"

gpu = True

num_layers = 2
embed_size = 100
hidden_size = 100
latent_size = 100

num_epochs=10

anneal = utils.kl_anneal_linear


In [ ]:
model = cvae.CVAE(len(DE.vocab), len(EN.vocab), embed_size, hidden_size, latent_size, num_layers)
if gpu:
    model.cuda()
print("Number of parameters: {}".format(utils.count_parameters(model)))

In [ ]:
train.train(model,model_name, train_iter, val_iter, DE, EN, anneal, num_epochs, gpu)

In [ ]:
perp, elbo, nre, kl = utils.eval_vae(model, val_iter, pad, gpu=gpu)
bleu = utils.test_multibleu(model, val_iter, EN, gpu=gpu)
print(perp, elbo, nre, kl, bleu)